In [16]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import PCA
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
import os
import numpy as np
import io
from PIL import Image

In [17]:
if SparkSession.builder.getOrCreate().sparkContext:
    SparkSession.builder.getOrCreate().sparkContext.stop()

spark = (SparkSession
             .builder
             .appName('test2-spark')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

In [18]:
PATH = os.getcwd()
PATH_Data = PATH+'/data/Test1'
PATH_Result = PATH+'/data/results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train
PATH_Data:   /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/Test1
PATH_Result: /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/results


In [19]:
sc = spark.sparkContext
spark

In [20]:
df = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [21]:
df.show()

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5602|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5599|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5597|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5594|[FF D8 FF E0 00 1...|
|file:

In [22]:
from pyspark.sql.types import ArrayType, IntegerType

def process_image(content):
    img = Image.open(io.BytesIO(content)).convert('L')
    return np.array(img).flatten().tolist()

# Déclaration de la fonction UDF
process_image_udf = udf(process_image, ArrayType(IntegerType()))

# Ajout d'une colonne 'features' en appliquant l'UDF
df_final = df.withColumn("features", process_image_udf(df.content))

# Affichage du DataFrame final
df_final.show()

+--------------------+-------------------+------+--------------------+--------------------+
|                path|   modificationTime|length|             content|            features|
+--------------------+-------------------+------+--------------------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5602|[FF D8 FF E0 00 1...|[255, 255,